# Shapenet Rendering
- Blender script source: https://github.com/panmari/stanford-shapenet-renderer

## Semantic Segmentation
If rendering a single object, we get a mask (interpretted as a binary segmentation map) for free. However, if we want to render multiple objects, either for rendering our target object into a 3D scene or for using secondary models to produce oclusions, then we need to do something more. 

Here are some references for how we can do semantic segmentation:
- https://blender.stackexchange.com/questions/79595/change-diffuse-shader-to-emission-shader-without-affecting-shader-color
- https://blender.stackexchange.com/questions/80906/create-a-segmentation-picture-with-each-object-class-rendered-in-different-color/162746#162746
- https://blender.stackexchange.com/questions/34609/is-there-a-way-to-streamline-scripting-these-shaders-and-modifier-keyframes
- https://github.com/DIYer22/bpycv#install
 - This seems promising, but I had issues when trying to install openexr

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import json
import glob
import cv2
import imageio
import shutil

import subprocess
import numpy as np

from modules.ShapeNetHandler import ShapeNetHandler
import modules.utils

from IPython.display import Image, display

# Load ShapeNet Handler

In [3]:
shapenet_root = "/hdd/mliuzzolino/ShapeNet/data/ShapeNetCore.v2"
shapenet_handler = ShapeNetHandler(shapenet_root)

## Print available categories by name
- Can use name to condition random sampling

In [4]:
# shapenet_handler.print_categories()

## Randomly Sample a filepath

In [5]:
temp_obj_filepath, temp_synset_id, temp_instance_id, temp_obj_name = shapenet_handler.sample_obj(category_name='car')

## Output Path

In [6]:
_OUTPUT_ROOT = 'render_output'

# Render

In [24]:
_N_VIEW_ANGLES = 72
_N_MODELS_TO_RENDER = 100
_RANDOM_SCALING = False
_OVERWRITE = True
_RENDER_ENGINE = 'BLENDER_RENDER' # 'CYCLES', 'BLENDER_RENDER'
_RENDER_SCRIPT = "scripts/blender_render.py"
_TARGET_CATEGORY_LIST = []#['airplane', 'car']

## Run Blender

In [ ]:
subprocess_outputs = []
try:
    for img_i in range(_N_MODELS_TO_RENDER):
        # Randomly select target category
        target_category = ''
        if _TARGET_CATEGORY_LIST != []:
            target_category = np.random.choice(_TARGET_CATEGORY_LIST)
        
        # Sample object filepath
        obj_filepath, synset_id, instance_id, obj_name = shapenet_handler.sample_obj(category_name=target_category)
        
        # Setup outpath
        output_root = os.path.join(_OUTPUT_ROOT, _RENDER_ENGINE, f'synsetID_{synset_id}', instance_id)
        
        # Randomly select obj scaling
        random_scale = np.random.uniform(0.99, 1.15) if _RANDOM_SCALING else 1.0
        random_n_occlusions = np.random.randint(3, 7)
        
        if not os.path.exists(output_root) or _OVERWRITE:
            # Remove previous directory if overwrite
            if _OVERWRITE and os.path.exists(output_root):
                shutil.rmtree(output_root)
            
            # Run blender script
            cmd = f"blender --background --python {_RENDER_SCRIPT} -- "
            cmd += f"--output_folder {output_root} --views {_N_VIEW_ANGLES} "
            cmd += f"--scale {random_scale} --n_occlusions {random_n_occlusions} "
            cmd += f"--render_engine {_RENDER_ENGINE} "
            cmd += f"{obj_filepath}"
            out = subprocess.check_output(cmd.split(" "), shell=False)
            subprocess_outputs.append(out)
        else:
            print("Skipping.")
        
        # Stdout
        stdout_str = f"[{img_i+1}/{_N_MODELS_TO_RENDER}] -- "
        stdout_str += f"SynsetID: {synset_id} -- "
        stdout_str += f"InstanceID: {instance_id:32s} -- "
        stdout_str += f"Name: {obj_name}"
        print(stdout_str)
        
except KeyboardInterrupt:
    try:
        # Try to cleanup directory with incomplete # renders
        if len(os.listdir(output_root)) != _N_VIEW_ANGLES:
            shutil.rmtree(output_root)
    except:
        pass
    print("\nEnding early.")

[1/100] -- SynsetID: 02942699 -- InstanceID: 68f66f5ab594fd3cc2890daf3a9f7413 -- Name: camera,photographic camera
[6/100] -- SynsetID: 03337140 -- InstanceID: 3711f7ae4f528f7963f393abca61510  -- Name: file,file cabinet,filing cabinet
[7/100] -- SynsetID: 04225987 -- InstanceID: 11aaf89382b3a4de983ec12a2b33f18b -- Name: skateboard
[8/100] -- SynsetID: 04074963 -- InstanceID: b2c79f7add7cb56ac03df0bb156692e3 -- Name: remote control,remote
[9/100] -- SynsetID: 03207941 -- InstanceID: 5d17e90f512a3dc7df3a1b0d597ce76e -- Name: dishwasher,dish washer,dishwashing machine
[10/100] -- SynsetID: 02801938 -- InstanceID: 3c06fb3595c39a4eeb6c29873b08c02  -- Name: basket,handbasket
[11/100] -- SynsetID: 03085013 -- InstanceID: 1e11af6fa598cd6960113b959388060d -- Name: computer keyboard,keypad
[12/100] -- SynsetID: 04554684 -- InstanceID: da0179a5b68f13586a6a687121d74e50 -- Name: washer,automatic washer,washing machine
[13/100] -- SynsetID: 04379243 -- InstanceID: affb5a80f11b383e1c25d54737ed5c8e -- 

## Background Images

In [19]:
background_imgs_root = '/hdd/mliuzzolino/Places2/places365_standard/train'
background_img_paths = glob.glob(f'{background_imgs_root}/*/*')

## Load Images and Animate

In [20]:
_LOAD_LIMIT = 18

limit_reached = False

all_imgs = []
rendered_instance_ids = glob.glob(f"{os.path.join(_OUTPUT_ROOT, _RENDER_ENGINE)}/*/*")
np.random.shuffle(rendered_instance_ids)
for rendered_instance_id in rendered_instance_ids:
    sys.stdout.write(f"\rProcessed model {len(all_imgs)+1}/{_LOAD_LIMIT}...")
    sys.stdout.flush()

    # Load paths
    target_img_paths = np.sort(glob.glob(f"{rendered_instance_id}/*_target.png"))
    occluded_img_paths = [ele.replace('_target', '_occluded') for ele in target_img_paths]
    semantic_seg_paths = [ele.replace('_target', '_semseg') for ele in target_img_paths]
    

    # Load images
    target_imgs = [modules.utils.read_image(img_path, parse_mask=True) for img_path in target_img_paths]
    occluded_imgs = [modules.utils.read_image(img_path) for img_path in occluded_img_paths]
    segmentation_imgs = [modules.utils.read_image(img_path) for img_path in semantic_seg_paths]

    # Ensure correct number of view angles present
    if len(occluded_imgs) != _N_VIEW_ANGLES:
        print(f"\t**Error: Insufficient angles! Actual: {len(occluded_imgs)} - Expected: {_N_VIEW_ANGLES}")
        continue

    # Setup background image
    background_img = modules.utils.sample_background_img(background_img_paths, shape=occluded_imgs[0].shape)

    # Apply background image
    target_imgs = [modules.utils.add_background(rgb_img, seg_img, background_img) 
                         for rgb_img, seg_img in target_imgs]
    occluded_imgs = [modules.utils.add_background(rgb_img, seg_img, background_img) 
                         for rgb_img, seg_img in zip(occluded_imgs, segmentation_imgs)]

    # Join rgb image with semantic seg image
    joined_imgs = [(target, occluded, mask) 
                       for target, occluded, mask in zip(target_imgs, occluded_imgs, segmentation_imgs)]

    # Combine images
    joined_imgs = modules.utils.combine_imgs(joined_imgs)
    all_imgs.append(joined_imgs)

    # Check limit
    if len(all_imgs) >= _LOAD_LIMIT:
        print("\nLimit reached. Ending early.")
        limit_reached = True
        break

Processed model 18/18...
Limit reached. Ending early.


In [21]:
# Stack all images
stacked_imgs = modules.utils.stack_all_imgs(all_imgs, nrow=3)

In [22]:
savepath = 'test.gif'
animation = modules.utils.toAnimation(stacked_imgs, figsize=(12,12), interval=100, savepath=savepath, fps=15)

Building animator...
Generating animation object...
Saving to test.gif...
Fin.


In [23]:
if savepath.endswith('gif'):
    display(Image(filename=savepath))